<a href="https://colab.research.google.com/github/salianbharat/Deep/blob/main/RNN_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pydantic import BaseModel


In [4]:
# Load the datasets
train_df = pd.read_csv('/content/twitter_training.csv')
val_df = pd.read_csv('/content/twitter_validation.csv')

In [7]:
!pip install fastapi uvicorn pydantic
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.4 MB/s eta 0:00:00


In [9]:
from fastapi import FastAPI
import uvicorn
from pyngrok import ngrok

In [10]:
# Check the structure of the datasets
print(train_df.head())
print(val_df.head())

   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     
   3364   Facebook Irrelevant  \
0   352     Amazon    Neutral   
1  8312  Microsoft   Negative   
2  4371      CS-GO   Negative   
3  4433     Google    Neutral   
4  6273       FIFA   Negative   

  I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣  
0

In [15]:
train_df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [16]:
print(train_df.columns)

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')


In [17]:
print(val_df.columns)

Index(['3364', 'Facebook', 'Irrelevant',
       'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣'],
      dtype='object')


In [20]:
# Display the column names of the training dataset
print("Training Dataset Columns:")
print(train_df.columns)

# Display the column names of the validation dataset
print("\nValidation Dataset Columns:")
print(val_df.columns)

Training Dataset Columns:
Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

Validation Dataset Columns:
Index(['3364', 'Facebook', 'Irrelevant',
       'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣'],
      dtype='object')


In [21]:
# Rename the columns for clarity
train_df.columns = ['index', 'topic', 'sentiment', 'text']
val_df.columns = ['index', 'topic', 'sentiment', 'text']

In [22]:
# Drop the index column as it's not needed for analysis
train_df = train_df.drop(columns=['index'])
val_df = val_df.drop(columns=['index'])

In [23]:
# Combine the training and validation datasets for preprocessing
df = pd.concat([train_df, val_df])

In [25]:
# Ensure all entries in the 'text' column are strings and handle missing values
df['text'] = df['text'].astype(str).fillna('')

In [26]:
# Preprocess the textual data
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'].values)
sequences = tokenizer.texts_to_sequences(df['text'].values)
padded_sequences = pad_sequences(sequences, maxlen=100)

In [27]:
# Encode sentiment labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['sentiment'].values)

In [28]:
# Split the combined dataset back into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [30]:
# Choose an RNN architecture (LSTM)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=100),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [31]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Train the RNN model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)])

Epoch 1/10
1892/1892 [==============================] - 222s 115ms/step - loss: -674.2815 - accuracy: 0.3016 - val_loss: -1932.8771 - val_accuracy: 0.3007
Epoch 2/10
1892/1892 [==============================] - 206s 109ms/step - loss: -4068.4531 - accuracy: 0.3015 - val_loss: -6632.5615 - val_accuracy: 0.3007
Epoch 3/10
1892/1892 [==============================] - 207s 109ms/step - loss: -9810.7188 - accuracy: 0.3015 - val_loss: -13421.2842 - val_accuracy: 0.3007
Epoch 4/10
1892/1892 [==============================] - 207s 110ms/step - loss: -17486.9648 - accuracy: 0.3015 - val_loss: -22060.5781 - val_accuracy: 0.3007
Epoch 5/10
1892/1892 [==============================] - 212s 112ms/step - loss: -26966.3320 - accuracy: 0.3015 - val_loss: -32490.2832 - val_accuracy: 0.3007
Epoch 6/10
1892/1892 [==============================] - 207s 109ms/step - loss: -38215.8867 - accuracy: 0.3015 - val_loss: -44683.0117 - val_accuracy: 0.3007
Epoch 7/10
1892/1892 [==============================] - 21

In [33]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy:.2f}')

473/473 [==============================] - 15s 32ms/step - loss: -111063.5000 - accuracy: 0.3007
Validation Accuracy: 0.30


In [35]:
# Fine-tuning the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=100),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)])

Epoch 1/10
1892/1892 [==============================] - 471s 246ms/step - loss: -789.8496 - accuracy: 0.3015 - val_loss: -2255.3291 - val_accuracy: 0.3007
Epoch 2/10
1892/1892 [==============================] - 466s 246ms/step - loss: -4778.9517 - accuracy: 0.3015 - val_loss: -7792.0483 - val_accuracy: 0.3007
Epoch 3/10
1892/1892 [==============================] - 463s 245ms/step - loss: -11481.8584 - accuracy: 0.3015 - val_loss: -15676.7646 - val_accuracy: 0.3007
Epoch 4/10
1892/1892 [==============================] - 459s 243ms/step - loss: -20413.5371 - accuracy: 0.3015 - val_loss: -25755.5273 - val_accuracy: 0.3007
Epoch 5/10
1892/1892 [==============================] - 465s 246ms/step - loss: -31498.8008 - accuracy: 0.3015 - val_loss: -37974.4805 - val_accuracy: 0.3007
Epoch 6/10
1892/1892 [==============================] - 467s 247ms/step - loss: -44662.3164 - accuracy: 0.3015 - val_loss: -52207.5352 - val_accuracy: 0.3007
Epoch 7/10
1892/1892 [==============================] - 4

In [36]:
# Save the model
model.save('/content/RNN_sentiment_model')

In [37]:
# Set up FastAPI app
app = FastAPI()

In [39]:
# Define input model
class TextInput(BaseModel):
    text: str

# Load the trained model
model = tf.keras.models.load_model('/content/RNN_sentiment_model')

# Define prediction endpoint
@app.post("/predict")
def predict_sentiment(input: TextInput):
    # Preprocess input text
    sequences = tokenizer.texts_to_sequences([input.text])
    padded_sequences = pad_sequences(sequences, maxlen=100)
    # Predict sentiment
    prediction = model.predict(padded_sequences)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return {"sentiment": sentiment}

# Define health check endpoint
@app.get("/health")
def health_check():
    return {"status": "ok"}

# Open a tunnel for the FastAPI app using ngrok
public_url = ngrok.connect(8000)
print("FastAPI public URL:", public_url)

# Run the FastAPI app
uvicorn.run(app, host='0.0.0.0', port=8000)

ERROR:pyngrok.process.ngrok:t=2024-07-17T16:42:23+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-17T16:42:23+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-17T16:42:23+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.